In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [6]:
df.shape

(569, 33)

In [7]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace = True)

In [8]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Train-Test Split

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size = 0.2)

Preprocessing

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [11]:
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
Y_test = encoder.fit_transform(Y_test)

In [13]:
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
Y_train = torch.tensor(Y_train)
Y_test = torch.tensor(Y_test)


Model

In [28]:
class SimpleNN():
    def __init__(self, X):
        self.weights = torch.rand((X.shape[1], 1), dtype = torch.float64, requires_grad = True)
        self.bias = torch.zeros(1, dtype = torch.float64, requires_grad = True)
    
    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bias
        return torch.sigmoid(z)
    
    def loss_function(self, y_pred, y):
        # Clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

        # Calculate loss
        loss = -(Y_train * torch.log(y_pred) + (1 - Y_train) * torch.log(1 - y_pred)).mean()
        return loss

In [29]:
lr = 0.1
epochs = 25

In [30]:
# create model
model = SimpleNN(X_train)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model.forward(X_train)

  # loss calculate
  loss = model.loss_function(y_pred, Y_train)

  # backward pass
  loss.backward()

  # parameters update
  with torch.no_grad():
    model.weights -= lr * model.weights.grad
    model.bias -= lr * model.bias.grad

  # zero gradients
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 3.371638529134447
Epoch: 2, Loss: 3.2401075227479392
Epoch: 3, Loss: 3.1033992122711016
Epoch: 4, Loss: 2.959288242383832
Epoch: 5, Loss: 2.8153703108050308
Epoch: 6, Loss: 2.666730607310275
Epoch: 7, Loss: 2.5165712874004504
Epoch: 8, Loss: 2.3625583297280097
Epoch: 9, Loss: 2.202669200009351
Epoch: 10, Loss: 2.045395854912877
Epoch: 11, Loss: 1.8899560111151263
Epoch: 12, Loss: 1.7371336415442218
Epoch: 13, Loss: 1.587644121715439
Epoch: 14, Loss: 1.446495171444577
Epoch: 15, Loss: 1.3180558169458303
Epoch: 16, Loss: 1.204135126258193
Epoch: 17, Loss: 1.1063056065664367
Epoch: 18, Loss: 1.0256220169489738
Epoch: 19, Loss: 0.9622815543236248
Epoch: 20, Loss: 0.9151693597348171
Epoch: 21, Loss: 0.881519222476858
Epoch: 22, Loss: 0.8575993829345573
Epoch: 23, Loss: 0.8400898496736584
Epoch: 24, Loss: 0.8266721345729386
Epoch: 25, Loss: 0.815876665978622


In [31]:
model.weights

tensor([[-0.3604],
        [ 0.4443],
        [ 0.3808],
        [ 0.3453],
        [-0.1385],
        [-0.4200],
        [-0.1835],
        [ 0.0371],
        [-0.1260],
        [ 0.2214],
        [ 0.0414],
        [ 0.0640],
        [-0.3600],
        [-0.3571],
        [ 0.4363],
        [-0.0947],
        [ 0.0939],
        [ 0.1907],
        [ 0.6369],
        [-0.1635],
        [ 0.1881],
        [ 0.1238],
        [ 0.2351],
        [ 0.4060],
        [ 0.4953],
        [-0.3579],
        [-0.4771],
        [ 0.0071],
        [ 0.2069],
        [ 0.3942]], dtype=torch.float64, requires_grad=True)

In [32]:
model.bias

tensor([-0.1805], dtype=torch.float64, requires_grad=True)

In [40]:
with torch.no_grad():
    y_pred = model.forward(X_test)
    y_pred = (y_pred > 0.9).float()
    accuracy = (y_pred == Y_test).float().mean()
    print(f"Accuracy:{accuracy.item()}")

Accuracy:0.5652508735656738
